In [16]:
using CSV, DataFrames, Statistics, Dates, Plots, LinearAlgebra, DecisionTree

In [18]:
using Pkg
Pkg.add("Plots")

   Resolving package versions...
  No Changes to `C:\Users\Antoine\.julia\environments\v1.10\Project.toml`
  No Changes to `C:\Users\Antoine\.julia\environments\v1.10\Manifest.toml`


In [3]:
trainData = CSV.read("./data/train.csv", DataFrame)
testData = CSV.read("./data/test.csv", DataFrame)
first(trainData, 5)

Row,annee,type,nombre_cylindres,cylindree,transmission,boite,consommation
,Int64,String31,Int64,String3,String15,String15,String31
1,2023,voiture_moyenne,8,"4,4",integrale,automatique,"13,8358823529412"
2,2020,VUS_petit,4,2,integrale,automatique,"9,80041666666667"
3,2021,voiture_compacte,6,"3,3",propulsion,automatique,"11,7605"
4,2023,voiture_deux_places,8,5,integrale,automatique,"13,0672222222222"
5,2022,voiture_moyenne,8,"4,4",integrale,automatique,"13,8358823529412"


In [4]:
trainData.consommation = parse.(Float64,replace.(trainData.consommation, "," => "."))
trainData.cylindree = parse.(Float64,replace.(trainData.cylindree, "," => ".")) 

testData.cylindree = parse.(Float64,replace.(testData.cylindree, "," => "."))

150-element Vector{Float64}:
 2.5
 2.5
 2.5
 2.0
 5.8
 5.0
 5.0
 2.4
 3.5
 5.2
 5.2
 5.9
 2.0
 ⋮
 1.6
 3.3
 5.0
 2.0
 2.0
 1.6
 2.0
 2.0
 3.0
 3.0
 1.5
 2.0

In [5]:
trainData[!,:volume_gaz] = trainData[!,:nombre_cylindres] .* trainData[!,:cylindree]

testData[!,:volume_gaz] = testData[!,:nombre_cylindres] .* testData[!,:cylindree]

150-element Vector{Float64}:
 10.0
 10.0
 10.0
  8.0
 46.4
 40.0
 40.0
  9.6
 21.0
 52.0
 52.0
 70.80000000000001
  8.0
  ⋮
  6.4
 19.799999999999997
 40.0
  8.0
  8.0
  6.4
  8.0
  8.0
 18.0
 18.0
  4.5
  8.0

In [6]:
X_train =  Matrix(trainData[:, Not(:consommation)])
y_train = trainData.consommation
X_test = Matrix(testData)

150×7 Matrix{Any}:
 2014  "voiture_moyenne"        …  "manuelle"     10.0
 2014  "voiture_moyenne"           "automatique"  10.0
 2014  "VUS_petit"                 "automatique"  10.0
 2014  "VUS_petit"                 "automatique"   8.0
 2014  "voiture_sous_compacte"     "manuelle"     46.4
 2014  "voiture_sous_compacte"  …  "automatique"  40.0
 2014  "voiture_sous_compacte"     "manuelle"     40.0
 2014  "VUS_petit"                 "automatique"   9.6
 2014  "VUS_petit"                 "automatique"  21.0
 2014  "voiture_deux_places"       "manuelle"     52.0
 2014  "voiture_deux_places"    …  "manuelle"     52.0
 2014  "voiture_sous_compacte"     "automatique"  70.8
 2014  "VUS_petit"                 "automatique"   8.0
    ⋮                                     ⋱  ⋮                        
 2023  "VUS_petit"                 "automatique"   6.4
 2023  "voiture_moyenne"           "automatique"  19.8
 2024  "voiture_deux_places"    …  "automatique"  40.0
 2024  "VUS_petit"            

In [7]:
function decisionTree_remise(X_train ,y_train,  X_test, max_depth)
    model = DecisionTreeRegressor(max_depth=max_depth)
    fit!(model, X_train, y_train) #peut etre normaliser sur tout les données, pas justr sur les données d'entrainement
    ychap =  predict(model, X_test)  
    return ychap
end

decisionTree_remise (generic function with 1 method)

In [9]:
res = decisionTree_remise(X_train ,y_train,  X_test, 5)
testData.consommation = res

150-element Vector{Float64}:
  8.773101155974718
  8.070047540540804
  9.545548529167164
  9.889100280579937
 14.682977190876372
 14.682977190876372
 14.682977190876372
  9.889100280579937
 10.924893508841617
 15.345064163165286
 15.345064163165286
 15.345064163165286
  8.840702060295868
  ⋮
  7.842333308965198
 11.632846725905672
 12.897491767520393
 10.686605970839567
  8.840702060295868
  7.842333308965198
  8.070047540540804
  8.845504273504272
 12.121046960836416
 10.620542993264337
  7.644047072551106
  8.070047540540804

In [21]:
id = 1:150

df_pred = DataFrame(id=id, consommation=res)

CSV.write("benchmark1.csv", df_pred)

"benchmark1.csv"